<a href="https://colab.research.google.com/github/Taniya-H/Email-Spam-Detection-ML/blob/main/Spam_Email_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'sms-spam-collection-dataset' dataset.
Path to dataset files: /kaggle/input/sms-spam-collection-dataset


In [43]:
import pandas as pd
import os
os.listdir("/kaggle/input/sms-spam-collection-dataset")

['spam.csv']

In [44]:
data = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding="latin-1")
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [45]:
data.head(6)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN


In [46]:
data.shape

(5572, 5)

In [47]:
data.isna().sum()

,0
v1,0
v2,0
Unnamed: 2,5522
Unnamed: 3,5560
Unnamed: 4,5566


In [48]:
data = data.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
data.rename(columns={'v1': 'Category', 'v2': 'Message'}, inplace=True)

In [49]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Category'] = le.fit_transform(data['Category'])

In [50]:
data

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [51]:
data.shape

(5572, 2)

In [52]:
data['Category'].value_counts()#0 is ham 1 is spam

,count
Category,
0,4825
1,747


In [53]:
from sklearn.model_selection import train_test_split

X = data['Message']
y = data['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [55]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(class_weight='balanced', max_iter=5600)
reg.fit(X_train_vec, y_train)

predict = reg.predict(X_test_vec)

from sklearn.metrics import classification_report
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.93      0.91      0.92       150

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [56]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

print("Accuracy:", accuracy_score(y_test, predict))
print("Spam precision:", precision_score(y_test, predict))   # class 1
print("Spam recall   :", recall_score(y_test, predict))      # class 1
print("Confusion matrix:\n", confusion_matrix(y_test, predict))

Accuracy: 0.9775784753363229
Spam precision: 0.9251700680272109
Spam recall   : 0.9066666666666666
Confusion matrix:
 [[954  11]
 [ 14 136]]


In [57]:
sample = ["Congratulations! You won 10,000 rupees. Click here now."]
sample_vec = vectorizer.transform(sample)
print(reg.predict(sample_vec))

[1]


In [58]:
sample2 = ["Hey, are we still meeting at 6pm?"]
sample2_vec = vectorizer.transform(sample2)
print(reg.predict(sample2_vec))

[0]
